<a href="https://colab.research.google.com/github/Breno-Giacoppini/Challange-3-DISRUPTIVE-ARCHITECTURES-IOT-IOB-GENERATIVE-IA/blob/main/previsao_vendas_roupas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Função para importar dados de um arquivo CSV
def importar_dados_csv(caminho_csv):
    try:
        dados_csv = pd.read_csv(caminho_csv, encoding='utf-8', sep=';')
        print("Arquivo CSV carregado com sucesso!")
        return dados_csv
    except FileNotFoundError:
        print("Erro: Arquivo CSV não encontrado.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocorreu um erro ao carregar o arquivo CSV: {e}")
        return pd.DataFrame()

# Função para coletar dados manualmente
def coleta_de_dados():
    vendas = []

    while True:
        # Escolha do modelo
        print("\nSelecione o modelo de roupa para informar a quantidade de vendas no primeiro período:")
        print("1 - Camiseta")
        print("2 - Calça")
        print("3 - Jaqueta")
        print("4 - Bermuda")
        print("5 - Outro")
        escolha_modelo = input("\nDigite o número correspondente ao modelo: ")

        if escolha_modelo == "1":
            modelo = "Camiseta"
        elif escolha_modelo == "2":
            modelo = "Calça"
        elif escolha_modelo == "3":
            modelo = "Jaqueta"
        elif escolha_modelo == "4":
            modelo = "Bermuda"
        elif escolha_modelo == "5":
            modelo = input("\nInforme o nome do modelo: ")
        else:
            print("Opção inválida. Tente novamente.")
            continue

        # Escolher o tamanho com opções predefinidas
        print(f"\nInforme o tamanho do(a) {modelo}:")
        print("1 - PP")
        print("2 - P")
        print("3 - M")
        print("4 - G")
        print("5 - GG")
        escolha_tamanho = input("\nDigite o número correspondente ao tamanho: ")

        if escolha_tamanho == "1":
            tamanho = "PP"
        elif escolha_tamanho == "2":
            tamanho = "P"
        elif escolha_tamanho == "3":
            tamanho = "M"
        elif escolha_tamanho == "4":
            tamanho = "G"
        elif escolha_tamanho == "5":
            tamanho = "GG"
        else:
            print("Opção inválida. Tente novamente.")
            continue

        # Coletar as vendas do primeiro e segundo período
        quantidade_periodo1 = int(input(f"\nInforme a quantidade de vendas para o primeiro período (ex: janeiro) de {modelo} (tamanho {tamanho}): "))
        vendas.append({'modelo': modelo, 'tamanho': tamanho, 'quantidade': quantidade_periodo1, 'periodo': 1})

        quantidade_periodo2 = int(input(f"\nInforme a quantidade de vendas para o segundo período (ex: fevereiro) de {modelo} (tamanho {tamanho}): "))
        vendas.append({'modelo': modelo, 'tamanho': tamanho, 'quantidade': quantidade_periodo2, 'periodo': 2})

        # Perguntar se deseja informar mais períodos
        adicionar_periodo = input("\nDeseja informar a quantidade de vendas para um período adicional? (s/n): ")
        periodo_atual = 3
        while adicionar_periodo.lower() == 's':
            quantidade = int(input(f"\nInforme a quantidade de vendas para o período {periodo_atual} de {modelo} (tamanho {tamanho}): "))
            vendas.append({'modelo': modelo, 'tamanho': tamanho, 'quantidade': quantidade, 'periodo': periodo_atual})
            periodo_atual += 1
            adicionar_periodo = input("\nDeseja informar mais algum período? (s/n): ")

        # Perguntar se deseja adicionar outro produto
        adicionar_produto = input("\nDeseja cadastrar outro produto? (s/n): ")
        if adicionar_produto.lower() != 's':
            break

    return pd.DataFrame(vendas)

# Função principal
def main():
    opcao = input("Deseja inserir os dados manualmente ou importar um arquivo CSV?\n1. Inserir manualmente\n2. Importar CSV\nDigite o número da opção escolhida: ")

    if opcao == "2":
        caminho_csv = input("Por favor, informe o caminho do arquivo CSV: ")
        dados_vendas = importar_dados_csv(caminho_csv)
    else:
        dados_vendas = coleta_de_dados()

    if len(dados_vendas['periodo'].unique()) < 2:
        print("Erro: É necessário informar dados de pelo menos dois períodos.")
        return

    # Codificação das variáveis categóricas
    encoder = OneHotEncoder(handle_unknown='ignore')  # Ignorar categorias desconhecidas
    X = encoder.fit_transform(dados_vendas[['modelo', 'tamanho', 'periodo']])
    y = dados_vendas['quantidade']

    # Treinamento do modelo de regressão (melhor para prever quantidades contínuas)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    modelo_ml = RandomForestRegressor()  # Alterado para regressão
    modelo_ml.fit(X_train, y_train)

    # Prevendo para o próximo período
    previsoes = []
    for (modelo, tamanho), group in dados_vendas.groupby(['modelo', 'tamanho']):
        ultimo_periodo = group['periodo'].max()
        proximo_periodo = pd.DataFrame({
            'modelo': [modelo],
            'tamanho': [tamanho],
            'periodo': [ultimo_periodo + 1]
        })
        X_novo_periodo = encoder.transform(proximo_periodo[['modelo', 'tamanho', 'periodo']])
        predicao = modelo_ml.predict(X_novo_periodo)[0]
        previsoes.append({'modelo': modelo, 'tamanho': tamanho, 'quantidade_predita': predicao})

    previsoes_df = pd.DataFrame(previsoes)

    # Exibir as previsões corrigidas
    print("\nPrevisão de vendas para o próximo mês:")
    for _, row in previsoes_df.iterrows():
        print(f"- {round(row['quantidade_predita'])} unidades de {row['modelo']} (tamanho {row['tamanho']}) no próximo mês.")

if __name__ == "__main__":
    main()
